In [101]:
from selenium import webdriver
from bs4 import BeautifulSoup
# import requests
import time

driver = webdriver.Chrome()  # Ensure chromedriver is installed and in your PATH
url = "https://www.central.co.th/en/mc-jeans?page=2"
driver.get(url)

time.sleep(5)

rendered_html = driver.page_source
soup = BeautifulSoup(rendered_html, "html.parser")

In [103]:
# Find all <div> with class "central_container"
parent_divs = soup.find_all("div", class_="central_container")
if len(parent_divs) > 1:  # Ensure the 7th <div> exists
    # Access the 7th <div>
    specific_parent_div = parent_divs[6]

    # Find the nested <div> with data-widget="hits" and id="product-list"
    product_list_div = specific_parent_div.find("div", {"data-widget": "hits", "id": "product-list"})
    if product_list_div:
        # Find <div class="ais-Hits">
        ais_hits_div = product_list_div.find("div", class_="ais-Hits")
        if ais_hits_div:
            # Find <ol class="ais-Hits-list">
            hits_list = ais_hits_div.find_all("div", class_="sliderItem productCard")

        else:
            print("<div class='ais-Hits'> not found.")
    else:
        print("<div data-widget='hits' id='product-list'> not found.")
else:
    print("There are fewer than 7 <div> tags with class 'central_container'.")

## Raw data

## get data

### product-sku, product-name, product-price

In [112]:
data_product = []
number_product = 1

In [113]:
data = hits_list[number_product].prettify()
data_attributes_0  = data.split("\n")[0].split("data-")
data_attributes_0

desired_keys_0 = ['product-sku','product-name'] #'product-price'
data_list_0 = [attr.replace('"','').rstrip().replace('=','===') for attr in data_attributes_0 if any(attr.startswith(key) for key in desired_keys_0)]
print(data_list_0)
data_product = data_product+data_list_0

["product-name===Women's Jeans slim MBDP18520 Black", 'product-sku===MKP0366150']


In [114]:
data_attributes_1 = data.split("\n")
# print(data_attributes_1)

# desired_keys = ['product-finalprice','product-originalprice','product-pricesave']

data_list_1 = [attr.replace(' ', '').replace('฿','') for attr in data_attributes_1 if attr.replace(' ', '').startswith("฿")]
data_list_1 = data_list_1[:3]

discount_percent= float(data_list_1[-1])*100/float(data_list_1[-2])

data_list_1.append(str(int(discount_percent)))
data_list_1

keys = ['product-final-price','product-original-price','product-discount-bath','product-discount-percent']
merged_list = [f"{key}==={value}" for key, value in zip(keys, data_list_1)]
print(merged_list)
data_product = data_product+merged_list

['product-final-price===1,295', 'product-original-price===2195', 'product-discount-bath===900', 'product-discount-percent===41']


### product-categories, product-config-sku

In [118]:
import re
import json

desired_keys_2 = ['product-categories','product-config-sku']

data_attributes_2  = data.split("\n")[0].split("data-")
data_list_2 = [attr for attr in data_attributes_2 if any(attr.startswith(key) for key in desired_keys_2)]
data_list_2

## product-categories
input_string = data_list_2[0].split(":")[-1]
cleaned_string = re.sub(r'[\'"\[\]{}]', '', input_string)
split_string = cleaned_string.split(' /// ')
result = ','.join(split_string).rstrip()
data_product.append(f'product-categories==={result}')
print(f'product-categories==={result}')

## product-config-sku
if len(data_list_2[1].split("'")) > 1:
    input_string = data_list_2[1].split("'")[1]
    parsed_dict = json.loads(input_string)
    transformed_dict = {f"size{v['size']}": {"sku": v["sku"]} for k, v in parsed_dict.items()}
    result = json.dumps(transformed_dict, separators=(',', ':')).strip('{}')+'}'
    data_product.append('product-config-sku==='+'{'+result+'}')
    print('product-config-sku==='+'{'+result+'}')
else:
    result = 'None'
    data_product.append('product-config-sku==='+result)
    print('product-config-sku==='+result)

product-categories===Women,Women Clothing,Women Pants,Women Skinny Pants
product-config-sku==={"size28 EU":{"sku":"MKP0366163"},"size27 EU":{"sku":"MKP0366150"}}


## get image

In [116]:
url = data.split("\n")[3].split('src=')[-1]
cleaned_url = url.strip('\'"').rstrip('/>').strip('"')
cleaned_url
data_product.append('product-image==='+cleaned_url)
print('product-image==='+cleaned_url)

product-image===https://assets.central.co.th/MCJEANS-WomensJeansslimMBDP18520Black-MKP0366150-1?$JPEG$&amp;wid=199


In [117]:
print(data_product)

["product-name===Women's Jeans slim MBDP18520 Black", 'product-sku===MKP0366150', 'product-final-price===1,295', 'product-original-price===2195', 'product-discount-bath===900', 'product-discount-percent===41', 'product-categories===Women,Women Clothing,Women Pants,Women Skinny Pants', 'product-config-sku==="size28 EU":{"sku":"MKP0366163"},"size27 EU":{"sku":"MKP0366150"}', 'product-image===https://assets.central.co.th/MCJEANS-WomensJeansslimMBDP18520Black-MKP0366150-1?$JPEG$&amp;wid=199']


## list to dict

In [120]:
# Convert to dictionary
attributes_dict = {}
for attribute in data_product:
    # print(attribute)
    key, value = attribute.split("===")  # Split at the '='
    attributes_dict[key.strip()] = value.strip().strip('"')  # Remove spaces and quotes

# Print the dictionary
print(attributes_dict)
attributes_dict



{'product-name': "Women's Jeans slim MBDP18520 Black", 'product-sku': 'MKP0366150', 'product-final-price': '1,295', 'product-original-price': '2195', 'product-discount-bath': '900', 'product-discount-percent': '41', 'product-categories': 'Women,Women Clothing,Women Pants,Women Skinny Pants', 'product-config-sku': '{"size28 EU":{"sku":"MKP0366163"},"size27 EU":{"sku":"MKP0366150"}}', 'product-image': 'https://assets.central.co.th/MCJEANS-WomensJeansslimMBDP18520Black-MKP0366150-1?$JPEG$&amp;wid=199'}


{'product-name': "Women's Jeans slim MBDP18520 Black",
 'product-sku': 'MKP0366150',
 'product-final-price': '1,295',
 'product-original-price': '2195',
 'product-discount-bath': '900',
 'product-discount-percent': '41',
 'product-categories': 'Women,Women Clothing,Women Pants,Women Skinny Pants',
 'product-config-sku': '{"size28 EU":{"sku":"MKP0366163"},"size27 EU":{"sku":"MKP0366150"}}',
 'product-image': 'https://assets.central.co.th/MCJEANS-WomensJeansslimMBDP18520Black-MKP0366150-1?$JPEG$&amp;wid=199'}

# -----------------------

In [123]:
import re
import json
from selenium import webdriver
from bs4 import BeautifulSoup
import time

def html_parser(url):
    driver = webdriver.Chrome()  # Ensure chromedriver is installed and in your PATH
    driver.get(url)
    
    time.sleep(5)
    rendered_html = driver.page_source
    soup = BeautifulSoup(rendered_html, "html.parser")
    return(soup)

    

def hits_list(soup):
    parent_divs = soup.find_all("div", class_="central_container")
    if len(parent_divs) > 1:  
    
        specific_parent_div = parent_divs[6]
        
        product_list_div = specific_parent_div.find("div", {"data-widget": "hits", "id": "product-list"})
        if product_list_div:
        
            ais_hits_div = product_list_div.find("div", class_="ais-Hits")
            if ais_hits_div:
                hits_list = ais_hits_div.find_all("div", class_="sliderItem productCard")

            else:
                print("<div class='ais-Hits'> not found.")
        else:
            print("<div data-widget='hits' id='product-list'> not found.")
    else:
        print("There are fewer than 7 <div> tags with class 'central_container'.")
    
    return(hits_list)

def get_data_0(hits_list):
    # data = hits_list[index_product].prettify()
    data_attributes_0  = hits_list.split("\n")[0].split("data-")
    data_attributes_0

    desired_keys_0 = ['product-sku','product-name'] #'product-price'
    data_list_0 = [attr.replace('"','').rstrip().replace('=','===') for attr in data_attributes_0 if any(attr.startswith(key) for key in desired_keys_0)]
    # print(data_list_0)
    # data_product = data_product+data_list_0
    return (data_list_0)
    
def get_data_1(hits_list):
    data_attributes_1 = hits_list.split("\n")
    # print(data_attributes_1)

    # desired_keys = ['product-finalprice','product-originalprice','product-pricesave']

    data_list_1 = [attr.replace(' ', '').replace('฿','') for attr in data_attributes_1 if attr.replace(' ', '').startswith("฿")]
    data_list_1 = data_list_1[:3]

    discount_percent= float(data_list_1[-1])*100/float(data_list_1[-2])

    data_list_1.append(str(int(discount_percent)))
    data_list_1

    keys = ['product-final-price','product-original-price','product-discount-bath','product-discount-percent']
    merged_list = [f"{key}==={value}" for key, value in zip(keys, data_list_1)]
    # print(merged_list)
    # data_product = data_product+merged_list
    return(merged_list)
    
def get_data_2(hits_list):
    data_list = []

    desired_keys_2 = ['product-categories','product-config-sku']

    data_attributes_2  = hits_list.split("\n")[0].split("data-")
    data_list_2 = [attr for attr in data_attributes_2 if any(attr.startswith(key) for key in desired_keys_2)]

    ## product-categories
    input_string = data_list_2[0].split(":")[-1]
    cleaned_string = re.sub(r'[\'"\[\]{}]', '', input_string)
    split_string = cleaned_string.split(' /// ')
    result = ','.join(split_string).rstrip()
    data_list.append(f'product-categories==={result}')
    # print(f'product-categories==={result}')

    ## product-config-sku
    if len(data_list_2[1].split("'")) > 1:
        input_string = data_list_2[1].split("'")[1]
        parsed_dict = json.loads(input_string)
        transformed_dict = {f"size{v['size']}": {"sku": v["sku"]} for k, v in parsed_dict.items()}
        result = json.dumps(transformed_dict, separators=(',', ':')).strip('{}')+'}'
        data_list.append('product-config-sku==='+'{'+result+'}')
    else:
        result = 'None'
        data_list.append('product-config-sku==='+result)
        print('product-config-sku: None')
    # print('product-config-sku==='+result)
    
    return(data_list)
    
def get_data_3(hits_list):
    url = hits_list.split("\n")[3].split('src=')[-1]
    cleaned_url = url.strip('\'"').rstrip('/>').strip('"')
    
    # data_product.append('product-image==='+cleaned_url)
    data_list = [f"product-image==={cleaned_url}"]
    # print('product-image==='+cleaned_url)
    return(data_list)
    
# def list_2_dict(data_list):
#     result_dict = {}
#     for item in data_list:
#         key, value = item.split('===', 1)  # Split into key and value
#         try:
#             # Attempt to parse value as JSON if possible
#             value = json.loads(value)
#         except json.JSONDecodeError:
#             pass  # Leave as string if not JSON
#         result_dict[key] = value

#     return result_dict

def list_2_dict(data_list):
    result_dict = {}
    for item in data_list:
        key, value = item.split("===")  # Split into key and value
        result_dict[key.strip()] = value.strip().strip('"')

    return result_dict
    
def schema_metadata(data):
    config_sku_str = data["product-config-sku"]
    config_sku_dict = json.loads("{" + config_sku_str + "}")  # Adding braces to make it valid JSON

    # Create the transformed structure
    transformed_data = {
        data["product-sku"]: {
            "product-final-price": data["product-final-price"],
            "product-original-price": data["product-original-price"],
            "product-discount-percent": data["product-discount-percent"],
            "product-config-sku": config_sku_dict,
            "product-image": data["product-image"]
        }
    }
    return(transformed_data)

def schema_vecterdata(data):
    config_sku_str = data["product-config-sku"]
    config_sku_dict = json.loads("{" + config_sku_str + "}")  # Adding braces to make it valid JSON

    # Create the transformed structure
    transformed_data = {
        "product-sku": data["product-final-price"],
        "embedding": None,
        "fields": {
            "product-name": data["product-name"],
            "product-categories": data["product-categories"]
            }
    }
    return(transformed_data)    

In [124]:
data_dict_result=[]
for page in range(1,4,1):
    url ="https://www.central.co.th/en/mc-jeans"
    url_page = f"https://www.central.co.th/en/mc-jeans?page={page}"
    print(f'Running ... : {url_page}')
    soup = html_parser(url_page)
    html_data = hits_list(soup)
    
    for index in range(len(html_data)):
        data_list=[]
        data = html_data[index].prettify()
        
        data_0 = get_data_0(data)
        data_list = data_list+data_0
        
        data_1 = get_data_1(data)
        data_list = data_list+data_1
        
        data_2 = get_data_2(data)
        data_list = data_list+data_2
        
        data_3 = get_data_3(data)
        data_list = data_list+data_3
        
        data_dict = list_2_dict(data_list)
        # print(data_dict)
        
        data_dict_result.append(data_dict)

    # data_dict_result
    # with open("output.json", "w") as json_file:
    #     json.dump(data_dict_result, json_file, indent=4)

Running ... : https://www.central.co.th/en/mc-jeans?page=1
Running ... : https://www.central.co.th/en/mc-jeans?page=2
product-config-sku: None
product-config-sku: None
product-config-sku: None
product-config-sku: None
product-config-sku: None
product-config-sku: None
product-config-sku: None
Running ... : https://www.central.co.th/en/mc-jeans?page=3


In [128]:
with open("product_data.json", "w") as json_file:
    json.dump(data_dict_result, json_file, indent=4)

## Chroma database

In [1]:
import json

with open('product_data.json', 'r') as file:
    products_data = json.load(file)

In [2]:
import chromadb
from sentence_transformers import SentenceTransformer

client = chromadb.Client()
model = SentenceTransformer('all-MiniLM-L6-v2')

collection_name = 'product_embeddings'
collection = client.create_collection(collection_name)

In [3]:
# Delete all documents in the collection  # Deletes everything in the collection

for product in products_data:
    text_to_embed = product["product-name"] + " for " + product["product-categories"]
    # print(text_to_embed)
    
    # Ensure the model returns a valid embedding
    embedding = model.encode(text_to_embed)
    embedding = embedding.tolist()
    # print(embedding)
    
    # Store embedding in Chroma
    collection.add(
        documents=[product["product-sku"]],
        embeddings=[embedding],  # Ensure embedding is a list of floats
        metadatas=[product],
        ids=[product["product-sku"]]
    )
    

In [13]:
# Querying: Search for similar products based on a query
query = "short pant for women"
query_embedding = model.encode(query)

# Retrieve most similar products
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=2  # Return top 2 results
)

# Display results
for result in results['documents']:
    print(result)

['MKP1036776', 'MKP1036796']


In [ ]:
# Retrieve the most relevant passages based on the query
def get_relevant_passage(query: str, db, n_results: int):
    results = db.query(query_texts=[query], n_results=n_results)
    return [doc[0] for doc in results['documents']]

query = "What is the AI Maturity Scale?"
relevant_text = get_relevant_passage(query, db, n_results=1)

# Construct a prompt for the generation model based on the query and retrieved data
def make_rag_prompt(query: str, relevant_passage: str):
    escaped_passage = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
    prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and
strike a friendly and conversational tone.
QUESTION: '{query}'
PASSAGE: '{escaped_passage}'

ANSWER:
"""
    return prompt

# Generate an answer using the Gemini Pro API
def generate_answer(prompt: str):
    gemini_api_key = os.getenv("GEMINI_API_KEY")
    if not gemini_api_key:
        raise ValueError("Gemini API Key not provided. Please provide GEMINI_API_KEY as an environment variable")
    genai.configure(api_key=gemini_api_key)
    model = genai.GenerativeModel('gemini-pro')
    result = model.generate_content(prompt)
    return result.text

# Construct the prompt and generate the answer
final_prompt = make_rag_prompt(query, "".join(relevant_text))
answer = generate_answer(final_prompt)
print(answer)

# Interactive function to process user input and generate an answer
def process_query_and_generate_answer():
    query = input("Please enter your query: ")
    if not query:
        print("No query provided.")
        return
    db = load_chroma_collection(db_path, db_name)
    relevant_text = get_relevant_passage(query, db, n_results=1)
    if not relevant_text:
        print("No relevant information found for the given query.")
        return
    final_prompt = make_rag_prompt(query, "".join(relevant_text))
    answer = generate_answer(final_prompt)
    print("Generated Answer:", answer)

# Invoke the function to interact with user
process_query_and_generate_answer()